In [1]:
#x01_populationStatistics

In [2]:
from chewc.chewc import *

In [3]:
n_chr = 7
n_loci = 1000
founder_pop_size = 2

G = Genome(n_chr, n_loci)
founder_pop = create_pop(G, create_random_pop(G, founder_pop_size))
T = Trait(G, founder_pop, target_mean=0.0, target_variance=1.0)

In [4]:

sim = BreedingSimulation(G, T, h2
                         =0.2, reps=2, pop_size=200, selection_fraction=0.5)


In [5]:
# Instantiate the model with flexible input size
model = GeneticFeatureExtractor(input_length)
output = model(ins)

NameError: name 'input_length' is not defined

In [ ]:

# Simulate 10 generations (replace this with your RL training loop)
for generation in range(10):
    # Placeholder: Generate random actions (you'll use your RL agent's policy here)
    actions = 30
    state, reward = sim.step(actions) 
    print(f"Generation {generation+1}: Avg Phenotype = {state:.2f}, Reward = {reward:.2f}")

sim.plot_history()

In [ ]:
alleles = sim.population.haplotypes
ind, ploid, nchr, nloci = alleles.shape

In [ ]:
alleles.shape

In [ ]:
ins = alleles.reshape(ind,2, nchr*nloci)

In [ ]:
ins.shape

In [ ]:
model(ins)

In [ ]:


# Example usage with different input sizes
input_length = 7000  # Change this to any length
pop_size = 1
# dummy_data = create_dummy_data(pop_size, 2, input_length)



In [ ]:


print(output.shape)  # Expected output shape: (10, 64)

# Define a simple training loop for demonstration
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Dummy target data for demonstration purposes
target = torch.randn(pop_size, 64)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = model(dummy_data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")